In [ ]:
"""
Planning scheduler using OR-Tools CP-SAT
- 15 employees, 4 weeks (5 working days/week) => 20 days
- Activities: "Téléphone", "Renseignement", "Dérogation", "Réclamation", "Impayés"
- Takes into account employees' vacation days
- Enforces daily minima and weekly per-employee maxima described by the user
- If an employee works >= 3 days in a week, they must have at least 3 different activities that week
- Prints up to 5 different feasible solutions

How to run:
1) Install OR-Tools: pip install ortools
2) python planning_ortools_scheduler.py

Customize:
- Edit the `vacations` dict to reflect your real vacation days per employee.
- Days are indexed 0..19 (week 0 day 0 = 0, ... week 3 day 4 = 19).

Note: if your vacation pattern makes minimum daily staffing impossible, the model is infeasible.
"""

from ortools.sat.python import cp_model

num_employees = 15
weeks = 4
days_per_week = 5
num_days = weeks * days_per_week
activities = ["Téléphone", "Renseignement", "Dérogation", "Réclamation", "Impayés"]
num_activities = len(activities)

# Minimums per day (can modify if needed)
required_per_day = {
    "Téléphone": 5,
    "Renseignement": 3,
    "Dérogation": 1,
    "Réclamation": 1,
    "Impayés": 1
}
req = [required_per_day[a] for a in activities]

# Weekly maxima per employee (per activity)
weekly_max = {
    "Téléphone": 2,
    "Renseignement": 2,
    "Dérogation": 1,
    "Réclamation": 1,
    "Impayés": 1
}
wk_max = [weekly_max[a] for a in activities]

# Example vacations dict (employee -> list of global day indices off 0..19)
vacations = {
    0: [2, 3],
    1: [5],
    2: [10, 11, 12],
}

model = cp_model.CpModel()

# x[e,d,a] boolean: employee e on day d does activity a
x = {}
for e in range(num_employees):
    for d in range(num_days):
        for a in range(num_activities):
            x[(e,d,a)] = model.NewBoolVar(f"x_e{e}_d{d}_a{a}")

# Each employee at most one activity per day; if vacation -> zero activity
for e in range(num_employees):
    for d in range(num_days):
        model.Add(sum(x[(e,d,a)] for a in range(num_activities)) <= 1)
        if e in vacations and d in vacations[e]:
            model.Add(sum(x[(e,d,a)] for a in range(num_activities)) == 0)

# Daily minima per activity
for d in range(num_days):
    for a in range(num_activities):
        model.Add(sum(x[(e,d,a)] for e in range(num_employees)) >= req[a])

# Weekly per-employee constraints and distinct-activities constraint
for e in range(num_employees):
    for w in range(weeks):
        days = [w*days_per_week + i for i in range(days_per_week)]
        for a in range(num_activities):
            model.Add(sum(x[(e,d,a)] for d in days) <= wk_max[a])

        worked = [model.NewBoolVar(f"worked_e{e}_w{w}_d{d}") for d in days]
        for i, d in enumerate(days):
            model.Add(sum(x[(e,d,a)] for a in range(num_activities)) >= worked[i])
            model.Add(sum(x[(e,d,a)] for a in range(num_activities)) <= worked[i] * 1000)

        worked_sum = model.NewIntVar(0, days_per_week, f"worked_sum_e{e}_w{w}")
        model.Add(worked_sum == sum(worked))

        works_at_least_3 = model.NewBoolVar(f"works_at_least_3_e{e}_w{w}")
        model.Add(worked_sum >= 3).OnlyEnforceIf(works_at_least_3)
        model.Add(worked_sum < 3).OnlyEnforceIf(works_at_least_3.Not())

        y = [model.NewBoolVar(f"y_e{e}_w{w}_a{a}") for a in range(num_activities)]
        for a in range(num_activities):
            model.Add(sum(x[(e,d,a)] for d in days) >= y[a])
            model.Add(sum(x[(e,d,a)] for d in days) <= y[a] * days_per_week)

        model.Add(sum(y) >= 3).OnlyEnforceIf(works_at_least_3)

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30.0
solver.parameters.random_seed = 0
solver.parameters.num_search_workers = 8

class SolutionsPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, x, num_employees, num_days, num_activities, activities, max_sols=5):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._x = x
        self._num_employees = num_employees
        self._num_days = num_days
        self._num_activities = num_activities
        self._activities = activities
        self._sol_count = 0
        self._max_sols = max_sols

    def OnSolutionCallback(self):
        self._sol_count += 1
        print("\n" + "="*60)
        print(f"SOLUTION #{self._sol_count}")
        for w in range(weeks):
            print(f"\nSemaine {w+1}:")
            week_days = [w*days_per_week + i for i in range(days_per_week)]
            for d in week_days:
                counts = {a: 0 for a in self._activities}
                for e in range(self._num_employees):
                    for a in range(self._num_activities):
                        if self.Value(self._x[(e,d,a)]) == 1:
                            counts[self._activities[a]] += 1
                print(f" Jour {d} -> " + ", ".join(f"{k}:{v}" for k,v in counts.items()))

            print("\n  Répartition par employé (activité par jour index dans la semaine 0..4; '-' = congé):")
            for e in range(self._num_employees):
                row = []
                for d in week_days:
                    act = "-"
                    for a in range(self._num_activities):
                        if self.Value(self._x[(e,d,a)]) == 1:
                            act = self._activities[a][0]
                    row.append(act)
                distinct = set()
                worked_days = 0
                for d in week_days:
                    for a in range(self._num_activities):
                        if self.Value(self._x[(e,d,a)]) == 1:
                            distinct.add(self._activities[a])
                            worked_days += 1
                print(f"   E{e:02d}: " + " ".join(row) + f"   (jours travaillés: {worked_days}, activités différentes: {len(distinct)})")
        if self._sol_count >= self._max_sols:
            print("\nMaximum de solutions atteint. Arrêt de la recherche.")
            self.StopSearch()

printer = SolutionsPrinter(x, num_employees, num_days, num_activities, activities, max_sols=5)

status = solver.SearchForAllSolutions(model, printer)
print("\nStatus du solveur:", solver.StatusName(status))
print("Solutions trouvées:", printer._sol_count)


Status du solveur: MODEL_INVALID
Solutions trouvées: 0


In [ ]:
from ortools.sat.python import cp_model

# -------------------------------
# Callback : affichage JOURS en LIGNES, EMPLOYÉS en COLONNES + écriture fichier
# -------------------------------
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, shift, num_employees, num_days, activities, limit=5, file_path="solutions.txt"):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shift = shift
        self._num_employees = num_employees
        self._num_days = num_days
        self._activities = activities
        self._solution_count = 0
        self._solution_limit = limit
        self._file_path = file_path

        # Réinitialise le fichier au début
        with open(self._file_path, "w", encoding="utf-8") as f:
            f.write("=== Solutions de planning OR-Tools ===\n\n")

    def on_solution_callback(self):
        self._solution_count += 1

        header = f"\n{'='*60}\n✅ Solution {self._solution_count}\n{'='*60}\n"
        lines = [header]

        # En-tête
        header_line = "Jour    |" + "".join(f" Emp{e:2} |" for e in range(self._num_employees))
        lines.append(header_line)
        lines.append("-" * len(header_line))

        # Lignes jour par jour
        for d in range(self._num_days):
            line = f"Jour {d:2} |"
            for e in range(self._num_employees):
                val = self.Value(self._shift[e, d])
                if val == ACTIVITY_OFF:
                    act = "   "
                else:
                    act = self._activities[val][:3]
                line += f" {act:3} |"
            lines.append(line)

        # Résumé
        lines.append("\nRésumé (jours travaillés par employé) :")
        summary = " ".join(
            f"Emp{e}: {sum(1 for d in range(self._num_days) if self.Value(self._shift[e, d]) != ACTIVITY_OFF):2}j"
            for e in range(self._num_employees)
        )
        lines.append(summary)

        output = "\n".join(lines)
        print(output)

        # Écrire aussi dans le fichier
        with open(self._file_path, "a", encoding="utf-8") as f:
            f.write(output + "\n")

        if self._solution_count >= self._solution_limit:
            print("\n🚫 Limite de solutions atteinte, arrêt de la recherche.")
            self.StopSearch()

    def solution_count(self):
        return self._solution_count


# -------------------------------
# Données et modèle
# -------------------------------
num_employees = 15
num_weeks = 4
days_per_week = 5
num_days = num_weeks * days_per_week
activities = ["Téléphone", "Renseignement", "Dérogation", "Réclamation", "Impayés"]
num_activities = len(activities)
ACTIVITY_OFF = -1

days_off = {
    0: [0],
    1: [5, 6],
    2: [11, 12],
    6: [1],
    8: [3],
}

model = cp_model.CpModel()

# Variables
shift = {}
for e in range(num_employees):
    for d in range(num_days):
        if d in days_off.get(e, []):
            shift[e, d] = model.NewIntVar(ACTIVITY_OFF, ACTIVITY_OFF, f'shift_{e}_{d}')
        else:
            shift[e, d] = model.NewIntVar(ACTIVITY_OFF, num_activities - 1, f'shift_{e}_{d}')

# Chaque employé travaille s’il n’est pas en congé
for e in range(num_employees):
    for d in range(num_days):
        if d not in days_off.get(e, []):
            model.Add(shift[e, d] != ACTIVITY_OFF)

# Booléens is_assigned[e, d, a]
is_assigned = {}
for e in range(num_employees):
    for d in range(num_days):
        for a in range(num_activities):
            b = model.NewBoolVar(f'is_{e}_{d}_{a}')
            is_assigned[e, d, a] = b
            model.Add(shift[e, d] == a).OnlyEnforceIf(b)
            model.Add(shift[e, d] != a).OnlyEnforceIf(b.Not())

# Contraintes quotidiennes
for d in range(num_days):
    model.Add(sum(is_assigned[e, d, 0] for e in range(num_employees)) >= 5)  # Téléphone
    model.Add(sum(is_assigned[e, d, 1] for e in range(num_employees)) >= 3)  # Renseignement
    for a in [2, 3, 4]:
        model.Add(sum(is_assigned[e, d, a] for e in range(num_employees)) >= 1)

# Contraintes hebdomadaires
for e in range(num_employees):
    for w in range(num_weeks):
        days_in_week = list(range(w * days_per_week, (w + 1) * days_per_week))
        worked_bools = []
        for d in days_in_week:
            b = model.NewBoolVar(f'worked_{e}_{w}_{d}')
            model.Add(shift[e, d] != ACTIVITY_OFF).OnlyEnforceIf(b)
            model.Add(shift[e, d] == ACTIVITY_OFF).OnlyEnforceIf(b.Not())
            worked_bools.append(b)

        worked_days = model.NewIntVar(0, days_per_week, f'total_worked_{e}_{w}')
        model.Add(worked_days == sum(worked_bools))

        act_counts = {}
        for a in range(num_activities):
            count_var = model.NewIntVar(0, days_per_week, f'act_count_{e}_{w}_{a}')
            model.Add(count_var == sum(is_assigned[e, d, a] for d in days_in_week))
            act_counts[a] = count_var

        model.Add(act_counts[0] <= 2)
        model.Add(act_counts[1] <= 2)
        model.Add(act_counts[2] <= 1)
        model.Add(act_counts[3] <= 1)
        model.Add(act_counts[4] <= 1)

        works_at_least_3 = model.NewBoolVar(f'works_ge3_{e}_{w}')
        model.Add(worked_days >= 3).OnlyEnforceIf(works_at_least_3)
        model.Add(worked_days <= 2).OnlyEnforceIf(works_at_least_3.Not())

        diff_act_bools = []
        for a in range(num_activities):
            b = model.NewBoolVar(f'has_act_{e}_{w}_{a}')
            model.Add(act_counts[a] >= 1).OnlyEnforceIf(b)
            model.Add(act_counts[a] == 0).OnlyEnforceIf(b.Not())
            diff_act_bools.append(b)
        num_diff = model.NewIntVar(0, num_activities, f'num_diff_{e}_{w}')
        model.Add(num_diff == sum(diff_act_bools))
        model.Add(num_diff >= 3).OnlyEnforceIf(works_at_least_3)

# -------------------------------
# Résolution avec callback
# -------------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30
solver.parameters.num_search_workers = 8

solution_printer = SolutionPrinter(shift, num_employees, num_days, activities, limit=5)
solver.SearchForAllSolutions(model, solution_printer)

print(f"\n✅ Nombre total de solutions affichées : {solution_printer.solution_count()}")
print("📁 Les solutions ont aussi été écrites dans 'solutions.txt'")
